In [5]:
pip install wget 

Note: you may need to restart the kernel to use updated packages.


In [6]:
import wget
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
wget.download('https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt')

100% [......................................................] 1115394 / 1115394

'input (2).txt'

In [7]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [8]:
print("Length of dataset in Characters: ", len(text))

Length of dataset in Characters:  1115394


In [9]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [10]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
vocab_size


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


65

In [11]:
# Create a mapping of characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
iots = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] 
decode = lambda l: ''.join([iots[i] for i in l])

In [12]:
print(encode("hii there"))
print(decode(encode("hii, there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii, there


In [13]:
import torch

In [14]:
data = torch.tensor(encode(text),dtype = torch.long)

print(data.shape, data.type())
print(data[:1000])

torch.Size([1115394]) torch.LongTensor
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
 

In [15]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [16]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [17]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context}, the target is {target}")

When input is tensor([18]), the target is 47
When input is tensor([18, 47]), the target is 56
When input is tensor([18, 47, 56]), the target is 57
When input is tensor([18, 47, 56, 57]), the target is 58
When input is tensor([18, 47, 56, 57, 58]), the target is 1
When input is tensor([18, 47, 56, 57, 58,  1]), the target is 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is 58


In [18]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

In [19]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(f"This is the IX:{ix[0]}")
    x = torch.stack([data[i:i+block_size] for i in ix])
    print(f"This is the x:{x[0][0]}")
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    print(f"This is the y:{y[0][0]}")
    
    return x,y

In [20]:
xb, yb = get_batch('train')

This is the IX:76049
This is the x:24
This is the y:43


In [21]:
print(data[951600:951608])

tensor([39, 52, 42,  8,  0,  0, 25, 13])


In [22]:
print(data[951601:951610])

tensor([52, 42,  8,  0,  0, 25, 13, 30, 21])


In [23]:
print("Inputs")
print(xb.shape)
print(xb)
print('Targets')
print(yb.shape)
print(yb)
print("---------")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b : t + 1]
        target = yb[b,t]
        print(f"When the Input is {context.tolist()}, the target is {target.tolist()}")

Inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
Targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
---------
When the Input is [[24, 43, 58, 5, 57, 1, 46, 43]], the target is 43
When the Input is [[24, 43, 58, 5, 57, 1, 46, 43], [44, 53, 56, 1, 58, 46, 39, 58]], the target is 58
When the Input is [[24, 43, 58, 5, 57, 1, 46, 43], [44, 53, 56, 1, 58, 46, 39, 58], [52, 58, 1, 58, 46, 39, 58, 1]], the target is 5
When the Input is [[24, 43, 58, 5, 57, 1, 46, 43], [44, 53, 56, 1, 58, 46, 39, 58], [52, 58, 1, 58, 46, 39, 58, 1], [25, 17, 27, 10, 0, 21, 1, 54]], the target is 57
When the Input is [[24, 43, 58, 5, 57, 1, 46, 43], [44, 53, 56, 1, 58, 46, 39, 58], [52, 58, 1, 58, 46, 39, 58, 1], [25, 17, 27, 

In [24]:
# Making the Bigram Language Model

import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, target = None):
        logits = self.token_embedding_table(idx)
        if target is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            target = target.view(B*T)
            loss = F.cross_entropy(logits,target)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:,-1, :]
            probs = F.softmax(logits, dim = 1)
            idx_next = torch.multinomial(probs, num_samples = 1)
            idx = torch.cat((idx, idx_next), dim = 1)

        return idx
        

In [25]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [26]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [27]:
# Creating a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [28]:
batch_size = 32
for steps in range(10000):
    xb,yb = get_batch('train')
    logits, loss = m(xb,yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
print(loss.item())

This is the IX:817891
This is the x:57
This is the y:6
This is the IX:702318
This is the x:43
This is the y:56
This is the IX:82638
This is the x:8
This is the y:1
This is the IX:948169
This is the x:1
This is the y:53
This is the IX:984635
This is the x:1
This is the y:61
This is the IX:474962
This is the x:51
This is the y:43
This is the IX:610954
This is the x:56
This is the y:43
This is the IX:228222
This is the x:1
This is the y:40
This is the IX:222077
This is the x:47
This is the y:52
This is the IX:982728
This is the x:1
This is the y:58
This is the IX:381011
This is the x:1
This is the y:41
This is the IX:331222
This is the x:42
This is the y:6
This is the IX:292407
This is the x:52
This is the y:58
This is the IX:269874
This is the x:19
This is the y:20
This is the IX:951637
This is the x:42
This is the y:1
This is the IX:72114
This is the x:1
This is the y:57
This is the IX:77802
This is the x:57
This is the y:43
This is the IX:627861
This is the x:57
This is the y:47
This i

In [29]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


lso br. ave aviasurf my, yxMPZI ivee iuedrd whar ksth y h bora s be hese, woweee; the! KI 'de, ulseecherd d o blllando;LUCEO, oraingofof win!
RIfans picspeserer hee tha,
TOFonk? me ain ckntoty ded. bo'llll st ta d:
ELIS me hurf lal y, ma dus pe athouo
BEY:! Indy; by s afreanoo adicererupa anse tecorro llaus a!
OLeneerithesinthengove fal amas trr
TI ar I t, mes, n IUSt my w, fredeeyove
THek' merer, dd
We ntem lud engitheso; cer ize helorowaginte the?
Thak orblyoruldvicee chot, p,
Bealivolde Th li


In [30]:
# MATHMATICAL TRICK FOR SELF ATTENTION MECHANISM
torch.manual_seed(1337)
B,T,C = 4, 8, 32
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 32])

In [31]:
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)


In [32]:
x[0]

tensor([[ 1.8077e-01, -6.9988e-02, -3.5962e-01, -9.1520e-01,  6.2577e-01,
          2.5510e-02,  9.5451e-01,  6.4349e-02,  3.6115e-01,  1.1679e+00,
         -1.3499e+00, -5.1018e-01,  2.3596e-01, -2.3978e-01, -9.2111e-01,
          1.5433e+00,  1.3488e+00, -1.3964e-01,  2.8580e-01,  9.6512e-01,
         -2.0371e+00,  4.9314e-01,  1.4870e+00,  5.9103e-01,  1.2603e-01,
         -1.5627e+00, -1.1601e+00, -3.3484e-01,  4.4777e-01, -8.0164e-01,
          1.5236e+00,  2.5086e+00],
        [-6.6310e-01, -2.5128e-01,  1.0101e+00,  1.2155e-01,  1.5840e-01,
          1.1340e+00, -1.1539e+00, -2.9840e-01, -5.0754e-01, -9.2392e-01,
          5.4671e-01, -1.4948e+00, -1.2057e+00,  5.7182e-01, -5.9735e-01,
         -6.9368e-01,  1.6455e+00, -8.0299e-01,  1.3514e+00, -2.7592e-01,
         -1.5108e+00,  2.1048e+00,  2.7630e+00, -1.7465e+00,  1.4516e+00,
         -1.5103e+00,  8.2115e-01, -2.1153e-01,  7.7890e-01,  1.5333e+00,
          1.6097e+00, -4.0323e-01],
        [-8.3447e-01,  5.9780e-01, -5.14

In [33]:
xbow[0]

tensor([[ 1.8077e-01, -6.9988e-02, -3.5962e-01, -9.1520e-01,  6.2577e-01,
          2.5510e-02,  9.5451e-01,  6.4349e-02,  3.6115e-01,  1.1679e+00,
         -1.3499e+00, -5.1018e-01,  2.3596e-01, -2.3978e-01, -9.2111e-01,
          1.5433e+00,  1.3488e+00, -1.3964e-01,  2.8580e-01,  9.6512e-01,
         -2.0371e+00,  4.9314e-01,  1.4870e+00,  5.9103e-01,  1.2603e-01,
         -1.5627e+00, -1.1601e+00, -3.3484e-01,  4.4777e-01, -8.0164e-01,
          1.5236e+00,  2.5086e+00],
        [-2.4116e-01, -1.6063e-01,  3.2526e-01, -3.9683e-01,  3.9208e-01,
          5.7976e-01, -9.9692e-02, -1.1702e-01, -7.3193e-02,  1.2198e-01,
         -4.0159e-01, -1.0025e+00, -4.8488e-01,  1.6602e-01, -7.5923e-01,
          4.2481e-01,  1.4972e+00, -4.7132e-01,  8.1860e-01,  3.4460e-01,
         -1.7740e+00,  1.2990e+00,  2.1250e+00, -5.7775e-01,  7.8881e-01,
         -1.5365e+00, -1.6947e-01, -2.7318e-01,  6.1334e-01,  3.6581e-01,
          1.5667e+00,  1.0527e+00],
        [-4.3893e-01,  9.2179e-02,  1.99

In [34]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [35]:
torch.manual_seed(1337)
a = torch.ones(3,3)
b = torch.randint(0, 10,(3,2)).float()
c = a @ b # MATRIX MULTIPLICATION

In [36]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x
xbow2[0]
torch.allclose(xbow, xbow2)

False

In [37]:
print("a=")
print(a)
print("b=")
print(b)
print("c=")
print(c)

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
b=
tensor([[5., 7.],
        [2., 0.],
        [5., 3.]])
c=
tensor([[12., 10.],
        [12., 10.],
        [12., 10.]])


In [38]:
# SELF ATTENTION MECHANISM
B,T,C = 4, 8, 32 # batch, time, channels
tril = torch.tril(torch.ones(T,T))
tril = tril.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(tril, dim = -1)
xbow3 = wei @ x
xbow3[0]

tensor([[ 1.8077e-01, -6.9988e-02, -3.5962e-01, -9.1520e-01,  6.2577e-01,
          2.5510e-02,  9.5451e-01,  6.4349e-02,  3.6115e-01,  1.1679e+00,
         -1.3499e+00, -5.1018e-01,  2.3596e-01, -2.3978e-01, -9.2111e-01,
          1.5433e+00,  1.3488e+00, -1.3964e-01,  2.8580e-01,  9.6512e-01,
         -2.0371e+00,  4.9314e-01,  1.4870e+00,  5.9103e-01,  1.2603e-01,
         -1.5627e+00, -1.1601e+00, -3.3484e-01,  4.4777e-01, -8.0164e-01,
          1.5236e+00,  2.5086e+00],
        [-2.4116e-01, -1.6063e-01,  3.2526e-01, -3.9683e-01,  3.9208e-01,
          5.7976e-01, -9.9692e-02, -1.1702e-01, -7.3193e-02,  1.2198e-01,
         -4.0159e-01, -1.0025e+00, -4.8488e-01,  1.6602e-01, -7.5923e-01,
          4.2481e-01,  1.4972e+00, -4.7132e-01,  8.1860e-01,  3.4460e-01,
         -1.7740e+00,  1.2990e+00,  2.1250e+00, -5.7775e-01,  7.8881e-01,
         -1.5365e+00, -1.6947e-01, -2.7318e-01,  6.1334e-01,  3.6581e-01,
          1.5667e+00,  1.0527e+00],
        [-4.3893e-01,  9.2179e-02,  1.99

In [39]:
xbow3.shape

torch.Size([4, 8, 32])

In [40]:
torch.allclose(xbow, xbow3)

False

In [ ]:
# Now let's see how a single head self attention mechanism works
# Also called a decoder block in the transformer architecture
torch.manual_seed(1337)
B, T, C = 4, 8, 32

x = torch.randn(B, T, C)

#SINGLE HEAD SELF ATTENTION MECHANISM

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias= False)

k = key(x)
q = query(x)
v = value(x)
wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---->( B, T , T)

tril = torch.tril(torch.ones(T,T))
wei =  wei.masked_fill(tril ==0, float('-inf')) #Delete this line to convert it to an encoder block
wei = F.softmax(wei, dim=-1)
out = wei @ v
out[0]


tensor([[-0.1571,  0.8801,  0.1615, -0.7824, -0.1429,  0.7468,  0.1007, -0.5239,
         -0.8873,  0.1907,  0.1762, -0.5943, -0.4812, -0.4860,  0.2862,  0.5710],
        [ 0.6764, -0.5477, -0.2478,  0.3143, -0.1280, -0.2952, -0.4296, -0.1089,
         -0.0493,  0.7268,  0.7130, -0.1164,  0.3266,  0.3431, -0.0710,  1.2716],
        [ 0.4823, -0.1069, -0.4055,  0.1770,  0.1581, -0.1697,  0.0162,  0.0215,
         -0.2490, -0.3773,  0.2787,  0.1629, -0.2895, -0.0676, -0.1416,  1.2194],
        [ 0.1971,  0.2856, -0.1303, -0.2655,  0.0668,  0.1954,  0.0281, -0.2451,
         -0.4647,  0.0693,  0.1528, -0.2032, -0.2479, -0.1621,  0.1947,  0.7678],
        [ 0.2510,  0.7346,  0.5939,  0.2516,  0.2606,  0.7582,  0.5595,  0.3539,
         -0.5934, -1.0807, -0.3111, -0.2781, -0.9054,  0.1318, -0.1382,  0.6371],
        [ 0.3428,  0.4960,  0.4725,  0.3028,  0.1844,  0.5814,  0.3824,  0.2952,
         -0.4897, -0.7705, -0.1172, -0.2541, -0.6892,  0.1979, -0.1513,  0.7666],
        [ 0.1866, -0.0